In [1]:
%load_ext autoreload
%autoreload 2

import torch 

from vlstm_parallel import vlstm_fw_torch
from vlstm_recurrent import vlstm_recurrent_sequence_stabilized
from vlstm_chunkwise_parallel import vlstm_chunkwise_parallel
from einops import rearrange
from torch.nn import functional as F


# Match vLSTM chunkwise parallel to parallel


In [2]:
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [3]:
B = 1
S = 12
NH = 1
DH = 2
EPS = 0.0

In [4]:
torch.manual_seed(5)
igate_preacts = torch.randn((B, NH, S, 1), dtype=DTYPE, device=DEVICE)
fgate_preacts = torch.randn((B, NH, S, 1), dtype=DTYPE, device=DEVICE)

In [5]:
igate_preacts = 5 * torch.arange(B * NH * S, dtype=DTYPE, device=DEVICE).reshape(B, NH, S, 1) / 10000
fgate_preacts = torch.arange(B * NH * S, dtype=DTYPE, device=DEVICE).reshape(B, NH, S, 1) +1 # / 100

In [6]:
# igate_preacts = 5 * torch.arange(B * NH * S, dtype=DTYPE, device=DEVICE).reshape(B, NH, S, 1) / 10000
# fgate_preacts = torch.arange(B * NH * S, dtype=DTYPE, device=DEVICE).reshape(B, NH, S, 1) / 100

In [7]:
# fgate_preacts, igate_preacts

In [8]:
qs = torch.randn((B, NH, S, DH), dtype=DTYPE, device=DEVICE)
ks = torch.randn((B, NH, S, DH), dtype=DTYPE, device=DEVICE)
vs = torch.randn((B, NH, S, DH), dtype=DTYPE, device=DEVICE)
vs.shape

torch.Size([1, 1, 12, 2])

In [9]:
y_p = vlstm_fw_torch(qs, ks, vs, igate_preacts, fgate_preacts, eps=EPS)

In [10]:
# y_r = vlstm_recurrent_sequence_stabilized(qs, ks, vs, igate_preacts, fgate_preacts, normalization_mode="max_abs_sum_C_1", eps=EPS)
# y_r, torch.allclose(y_p, y_r, atol=1e-5)

In [11]:
y_p

tensor([[[[-0.1036, -0.1770],
          [ 0.3631, -0.0846],
          [ 0.0722,  0.5899],
          [ 0.2939, -0.5616],
          [-0.9256, -0.9527],
          [-0.1738,  2.1624],
          [ 0.0913,  1.3176],
          [-0.4555, -0.8872],
          [ 0.9848,  1.8037],
          [ 0.5587,  2.6535],
          [ 0.2654, -1.3075],
          [ 0.2538,  4.0591]]]], device='cuda:0')

In [12]:
y_cp = vlstm_chunkwise_parallel(qs, ks, vs, igate_preacts, fgate_preacts, chunk_size=4)
y_cp

/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[[[-0.1036, -0.1770],
          [ 0.3631, -0.0846],
          [ 0.0722,  0.5899],
          [ 0.2939, -0.5616],
          [-0.9256, -0.9527],
          [-0.1738,  2.1624],
          [ 0.0913,  1.3176],
          [-0.4555, -0.8872],
          [ 0.9848,  1.8037],
          [ 0.5587,  2.6535],
          [ 0.2654, -1.3075],
          [ 0.2538,  4.0591]]]], device='cuda:0')

In [13]:
y_p - y_cp

tensor([[[[ 0.0000e+00,  0.0000e+00],
          [ 2.9802e-08, -2.2352e-08],
          [-2.2352e-08,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00],
          [ 5.9605e-08,  1.1921e-07],
          [-4.4703e-08, -4.7684e-07],
          [ 1.4901e-08, -1.1921e-07],
          [-5.9605e-08, -1.7881e-07],
          [ 1.1921e-07, -2.3842e-07],
          [-1.7881e-07,  0.0000e+00],
          [ 0.0000e+00,  2.3842e-07],
          [-1.4901e-07,  4.7684e-07]]]], device='cuda:0')